In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

data_dir = "Bean_Dataset/Bean_Dataset"
img_height, img_width = 128, 128
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    data_dir, target_size=(img_height, img_width),
    batch_size=batch_size, class_mode='categorical', subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir, target_size=(img_height, img_width),
    batch_size=batch_size, class_mode='categorical', subset='validation'
)

num_classes = len(train_gen.class_indices)

def build_dnn():
    model = Sequential([
        Flatten(input_shape=(img_height, img_width, 3)),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_cnn():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

dnn_model = build_dnn()
dnn_history = dnn_model.fit(train_gen, epochs=20, validation_data=val_gen)

cnn_model = build_cnn()
cnn_history = cnn_model.fit(train_gen, epochs=20, validation_data=val_gen)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(dnn_history.history['accuracy'], label='DNN Train')
plt.plot(dnn_history.history['val_accuracy'], label='DNN Val')
plt.plot(cnn_history.history['accuracy'], label='CNN Train')
plt.plot(cnn_history.history['val_accuracy'], label='CNN Val')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(dnn_history.history['loss'], label='DNN Train')
plt.plot(dnn_history.history['val_loss'], label='DNN Val')
plt.plot(cnn_history.history['loss'], label='CNN Train')
plt.plot(cnn_history.history['val_loss'], label='CNN Val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(f"DNN Train Acc: {dnn_history.history['accuracy'][-1]:.4f}, Val Acc: {dnn_history.history['val_accuracy'][-1]:.4f}")
print(f"CNN Train Acc: {cnn_history.history['accuracy'][-1]:.4f}, Val Acc: {cnn_history.history['val_accuracy'][-1]:.4f}")